In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from joblib import Parallel, delayed
import sys
sys.path.append('../')
from packages import actv_analysis
from packages.svm import SVM_fit
from packages.load_csv import units_for_svm

# Parameters
relus = range(2,6)
epochs = np.arange(0, 91, 10)
exps = np.arange(1, 11)
num_units = 100
rate_threshold = 0.05

In [ ]:
def update_response_rate(units, actv, subset=False):
    non_zero_counts = np.count_nonzero(actv, axis=(1, 2))

    # Get the total number of elements in the second and third dimensions
    total_elements = actv.shape[1] * actv.shape[2]

    # Compute the response rate for each sample
    response_rates = (non_zero_counts / total_elements) * 100
    
    for i in range(len(units)):
        if subset:
            units[i].response_rate_subset = response_rates[i]
        else:
            units[i].response_rate = response_rates[i]
    
    return units

## update response rate

In [ ]:
min_sz_idx=3; max_sz_idx=9; numbers=np.arange(2,21,2); inst=500
take = np.arange(0,100).reshape(10,10)[:,min_sz_idx:max_sz_idx+1].reshape(len(numbers)*(max_sz_idx-min_sz_idx+1))

for net in range(1,11):
    for relu in range(2,6):
        for epoch in range(0,91,10):
            pkl_filename = f'pkl/network{net}_Relu{relu}_epoch{epoch}.pkl'
            if os.path.exists(pkl_filename):
                try:
                    with open(pkl_filename, 'rb') as f:
                        units = pickle.load(f)
                    print("Pickle file found.")
                    actv_net = actv_analysis.get_actv_net(net=net, relu=relu, epoch=epoch)
                    actv_szAtoB = actv_net[:,take,:]
                    units = update_response_rate(units, actv_net, subset=False)
                    units = update_response_rate(units, actv_szAtoB, subset=True)
                    # Saving the updated units at the end of processing
                    with open(pkl_filename, 'wb') as f:
                        pickle.dump(units, f)
                    print(f'Successfully saved the updated units to {pkl_filename}')
                except Exception as e:
                    print(f"Pickle file {pkl_filename} failed to load due to error: {e}")

## update monotonicity

In [ ]:
min_sz_idx=3; max_sz_idx=9; numbers=np.arange(2,21,2); inst=500
take = np.arange(0,100).reshape(10,10)[:,min_sz_idx:max_sz_idx+1].reshape(len(numbers)*(max_sz_idx-min_sz_idx+1))

for net in range(1,11):
    for relu in range(2,6):
        for epoch in range(0,91,10):
            pkl_filename = f'pkl/network{net}_Relu{relu}_epoch{epoch}.pkl'
            if os.path.exists(pkl_filename):
                try:
                    with open(pkl_filename, 'rb') as f:
                        units = pickle.load(f)
                    print("Pickle file found.")
                    actv_net = actv_analysis.get_actv_net(net=net, relu=relu, epoch=epoch)
                    actv_analysis.update_monotonicity(units, actv_net, subset=True)
                    # Saving the updated units at the end of processing
                    with open(pkl_filename, 'wb') as f:
                        pickle.dump(units, f)
                    print(f'Successfully saved the updated units to {pkl_filename}')
                except Exception as e:
                    print(f"Pickle file {pkl_filename} failed to load due to error: {e}")

### Update spearman r (a metric for monotonicity)

In [3]:
min_nidx=1; max_nidx=9
min_sz_idx=3; max_sz_idx=9; numbers=np.arange(2,21,2); inst=500
take = np.arange(0,100).reshape(10,10)[min_nidx:max_nidx+1,min_sz_idx:max_sz_idx+1].reshape((max_nidx-min_nidx+1)*(max_sz_idx-min_sz_idx+1))

for net in range(1,11):
    for relu in range(4,5):
        for epoch in range(0,91,10):
            pkl_filename = f'pkl/4to20/network{net}_Relu{relu}_epoch{epoch}_4to20.pkl'
            if os.path.exists(pkl_filename):
                try:
                    with open(pkl_filename, 'rb') as f:
                        units = pickle.load(f)
                    print("Pickle file found.")
                    actv_net = actv_analysis.get_actv_net(net=net, relu=relu, epoch=epoch)
                    actv_analysis.update_spearmanr(units, actv_net, min_nidx=min_nidx,max_nidx=max_nidx, subset=True)
                    # Saving the updated units at the end of processing
                    with open(pkl_filename, 'wb') as f:
                        pickle.dump(units, f)
                    print(f'Successfully saved the updated units to {pkl_filename}')
                except Exception as e:
                    print(f"Pickle file {pkl_filename} failed to load due to error: {e}")

Pickle file found.
--- 177.99186396598816 seconds ---


/Users/dongillee/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch0_4to20.pkl
Pickle file found.
--- 121.06500220298767 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch10_4to20.pkl
Pickle file found.
--- 127.72975206375122 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch20_4to20.pkl
Pickle file found.
--- 86.0659351348877 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch30_4to20.pkl
Pickle file found.
--- 115.14956378936768 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch40_4to20.pkl
Pickle file found.
--- 150.63008618354797 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch50_4to20.pkl
Pickle file found.
--- 150.7542381286621 seconds ---
Successfully saved the updated units to pkl/4to20/network1_Relu4_epoch60_4to20.pkl
Pickle file found.
--- 167.76409029960632 seconds ---
Successfully saved the updated units to pkl/4

### update PN & PS

In [ ]:
min_nidx=1
max_nidx=9
min_sz_idx=3
max_sz_idx=9

for net in range(1,11):
    for relu in range(2,6):
        for epoch in range(0,91,10):
            print(f'net{net} relu{relu} epoch{epoch}')
            pkl_filename = f'pkl/network{net}_Relu{relu}_epoch{epoch}_4to20.pkl'
            if os.path.exists(pkl_filename):
                try:
                    with open(pkl_filename, 'rb') as f:
                        units = pickle.load(f)
                    print("Pickle file found.")
                    actv_net = actv_analysis.get_actv_net(net=net, relu=relu, epoch=epoch)
                    units = actv_analysis.update_PNS(units, actv_net, numbers=np.arange(2,21,2), sizes = np.arange(4,14), min_nidx=min_nidx, max_nidx=max_nidx, min_sz_idx=min_sz_idx, max_sz_idx=max_sz_idx)
                    # Saving the updated units at the end of processing
                    with open(pkl_filename, 'wb') as f:
                        pickle.dump(units, f)
                    print(f'Successfully saved the updated units to {pkl_filename}')
                except Exception as e:
                    print(f"Pickle file {pkl_filename} failed to load due to error: {e}")